# Section 13: SCIM (System for Cross-domain Identity Management) - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 13: SCIM** of the VeevaVault API.

SCIM (System for Cross-domain Identity Management) is designed to make managing user identities in cloud-based applications and services easier. Vault API is based on SCIM 2.0.

## 🎯 Endpoints to Test

### Discovery Endpoints
1. **Retrieve SCIM Provider** - `GET /api/{version}/scim/v2/ServiceProviderConfig`
2. **Retrieve SCIM Resource Types** - `GET /api/{version}/scim/v2/ResourceTypes`
3. **Retrieve SCIM Schemas** - `GET /api/{version}/scim/v2/Schemas`

### User Management
4. **Retrieve SCIM Users** - `GET /api/{version}/scim/v2/Users`
5. **Retrieve SCIM User** - `GET /api/{version}/scim/v2/Users/{user_id}`
6. **Create SCIM User** - `POST /api/{version}/scim/v2/Users`
7. **Update SCIM User** - `PUT /api/{version}/scim/v2/Users/{user_id}`
8. **Delete SCIM User** - `DELETE /api/{version}/scim/v2/Users/{user_id}`

### Group Management
9. **Retrieve SCIM Groups** - `GET /api/{version}/scim/v2/Groups`
10. **Retrieve SCIM Group** - `GET /api/{version}/scim/v2/Groups/{group_id}`
11. **Create SCIM Group** - `POST /api/{version}/scim/v2/Groups`
12. **Update SCIM Group** - `PUT /api/{version}/scim/v2/Groups/{group_id}`
13. **Delete SCIM Group** - `DELETE /api/{version}/scim/v2/Groups/{group_id}`

### 🔬 Testing Strategy
- **Discovery First**: Query SCIM provider configuration and schemas
- **Read-Only Focus**: Prioritize discovery and retrieval operations
- **Safe Operations**: Avoid creating/modifying SCIM resources in production
- **Comprehensive Coverage**: Test complete SCIM 2.0 compliance

---

In [1]:
# Essential imports for comprehensive SCIM API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime
import pandas as pd

# VeevaVault Core Client
from veevavault.client import VaultClient

# Essential Services for SCIM testing
from veevavault.services.authentication import AuthenticationService
from veevavault.services.queries import QueryService

print("✅ Essential imports loaded for SCIM API testing")

✅ Essential imports loaded for SCIM API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced API testing framework for SCIM operations.

In [3]:
# Enhanced baseline API testing framework for SCIM operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedSCIMAPITester:
    """Enhanced API testing framework with SCIM-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # SCIM-specific tracking
        self.scim_provider_config = None
        self.scim_schemas = []
        self.scim_resource_types = []
        self.scim_users = []
        self.scim_groups = []
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': f'Bearer {self.session_id}',  # SCIM uses Bearer tokens
            'Accept': 'application/scim+json',  # SCIM content type
            'Content-Type': 'application/scim+json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def cleanup_scim_operations(self):
        """Clean up any SCIM operations performed during testing"""
        # Note: In production, implement proper SCIM resource cleanup
        # For testing, we focus on read-only operations
        return {"cleanup_performed": "read_only_operations_only"}

print("✅ Enhanced SCIM API testing framework initialized")

✅ Enhanced SCIM API testing framework initialized


## 🧪 Comprehensive SCIM Tester

Creating a specialized tester for SCIM operations with comprehensive endpoint coverage.

In [4]:
class SCIMTester(EnhancedSCIMAPITester):
    """Comprehensive SCIM API tester"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def test_scim_discovery(self):
        """Test SCIM discovery endpoints"""
        discovery_endpoints = [
            (f"/api/{self.api_version}/scim/v2/ServiceProviderConfig", "SCIM Provider Configuration"),
            (f"/api/{self.api_version}/scim/v2/ResourceTypes", "SCIM Resource Types"),
            (f"/api/{self.api_version}/scim/v2/Schemas", "SCIM Schemas")
        ]
        
        for endpoint, test_name in discovery_endpoints:
            try:
                start_time = time.time()
                result = self.make_authenticated_request(endpoint)
                response_time = time.time() - start_time
                
                if result.success:
                    # Store configuration data
                    if "ServiceProviderConfig" in endpoint:
                        self.scim_provider_config = result.response_data
                    elif "ResourceTypes" in endpoint:
                        self.scim_resource_types = result.response_data.get('Resources', []) if result.response_data else []
                    elif "Schemas" in endpoint:
                        self.scim_schemas = result.response_data.get('Resources', []) if result.response_data else []
                    
                    api_result = APITestResult(
                        endpoint=endpoint,
                        method='GET',
                        success=True,
                        response_time=response_time,
                        test_name=test_name,
                        details={
                            'status_code': result.status_code,
                            'has_response_data': bool(result.response_data),
                            'data_type': type(result.response_data).__name__ if result.response_data else 'None'
                        }
                    )
                    self.results.append(api_result)
                    print(f"✅ {test_name}: SUCCESS ({response_time:.2f}s)")
                else:
                    api_result = APITestResult(
                        endpoint=endpoint,
                        method='GET',
                        success=False,
                        response_time=response_time,
                        test_name=test_name,
                        error_message=f"Status: {result.status_code}"
                    )
                    self.results.append(api_result)
                    print(f"❌ {test_name}: FAILED ({response_time:.2f}s) - Status: {result.status_code}")
                    
            except Exception as e:
                api_result = APITestResult(
                    endpoint=endpoint,
                    method='GET',
                    success=False,
                    response_time=time.time() - start_time if 'start_time' in locals() else 0,
                    test_name=test_name,
                    error_message=str(e)
                )
                self.results.append(api_result)
                print(f"❌ {test_name}: EXCEPTION - {str(e)}")
    
    def test_scim_users(self):
        """Test SCIM user management endpoints"""
        try:
            start_time = time.time()
            
            # Test retrieving all SCIM users
            users_endpoint = f"/api/{self.api_version}/scim/v2/Users"
            users_result = self.make_authenticated_request(users_endpoint)
            response_time = time.time() - start_time
            
            if users_result.success:
                users_data = users_result.response_data.get('Resources', []) if users_result.response_data else []
                self.scim_users = users_data
                
                result = APITestResult(
                    endpoint=users_endpoint,
                    method='GET',
                    success=True,
                    response_time=response_time,
                    test_name='SCIM Users Retrieval',
                    details={
                        'total_users': len(users_data),
                        'has_users': bool(users_data),
                        'sample_user_ids': [user.get('id') for user in users_data[:3]] if users_data else []
                    }
                )
                self.results.append(result)
                print(f"✅ SCIM Users Retrieval: SUCCESS ({response_time:.2f}s)")
                print(f"👥 Found {len(users_data)} SCIM users")
            else:
                result = APITestResult(
                    endpoint=users_endpoint,
                    method='GET',
                    success=False,
                    response_time=response_time,
                    test_name='SCIM Users Retrieval',
                    error_message=f"Status: {users_result.status_code}"
                )
                self.results.append(result)
                print(f"❌ SCIM Users Retrieval: FAILED ({response_time:.2f}s) - Status: {users_result.status_code}")
                
        except Exception as e:
            result = APITestResult(
                endpoint='/api/{version}/scim/v2/Users',
                method='GET',
                success=False,
                response_time=time.time() - start_time if 'start_time' in locals() else 0,
                test_name='SCIM Users Retrieval',
                error_message=str(e)
            )
            self.results.append(result)
            print(f"❌ SCIM Users Retrieval: EXCEPTION - {str(e)}")
    
    def test_scim_groups(self):
        """Test SCIM group management endpoints"""
        try:
            start_time = time.time()
            
            # Test retrieving all SCIM groups
            groups_endpoint = f"/api/{self.api_version}/scim/v2/Groups"
            groups_result = self.make_authenticated_request(groups_endpoint)
            response_time = time.time() - start_time
            
            if groups_result.success:
                groups_data = groups_result.response_data.get('Resources', []) if groups_result.response_data else []
                self.scim_groups = groups_data
                
                result = APITestResult(
                    endpoint=groups_endpoint,
                    method='GET',
                    success=True,
                    response_time=response_time,
                    test_name='SCIM Groups Retrieval',
                    details={
                        'total_groups': len(groups_data),
                        'has_groups': bool(groups_data),
                        'sample_group_ids': [group.get('id') for group in groups_data[:3]] if groups_data else []
                    }
                )
                self.results.append(result)
                print(f"✅ SCIM Groups Retrieval: SUCCESS ({response_time:.2f}s)")
                print(f"👥 Found {len(groups_data)} SCIM groups")
            else:
                result = APITestResult(
                    endpoint=groups_endpoint,
                    method='GET',
                    success=False,
                    response_time=response_time,
                    test_name='SCIM Groups Retrieval',
                    error_message=f"Status: {groups_result.status_code}"
                )
                self.results.append(result)
                print(f"❌ SCIM Groups Retrieval: FAILED ({response_time:.2f}s) - Status: {groups_result.status_code}")
                
        except Exception as e:
            result = APITestResult(
                endpoint='/api/{version}/scim/v2/Groups',
                method='GET',
                success=False,
                response_time=time.time() - start_time if 'start_time' in locals() else 0,
                test_name='SCIM Groups Retrieval',
                error_message=str(e)
            )
            self.results.append(result)
            print(f"❌ SCIM Groups Retrieval: EXCEPTION - {str(e)}")

print("✅ Comprehensive SCIM Tester ready")

✅ Comprehensive SCIM Tester ready


## 🔐 Initialize Testing and Authentication

Setting up the SCIM tester and performing authentication.

In [5]:
# Initialize the SCIM tester
print("🔧 Initializing SCIM tester...")

scim_tester = SCIMTester(
    vault_url=VAULT_URL,
    username=VAULT_USERNAME,
    password=VAULT_PASSWORD
)

print("✅ SCIM tester initialized")

🔧 Initializing SCIM tester...
🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ SCIM tester initialized


In [6]:
# Test 01: Authentication
print("🔐 Testing authentication...")
print("=" * 60)

auth_success = scim_tester.authenticate()

if auth_success:
    print(f"\n🎉 Authentication Successful!")
    print(f"🆔 Session ID: {scim_tester.session_id[:20]}...")
    print(f"🏢 Vault ID: {scim_tester.vault_id}")
    print(f"\n✅ Session established: {scim_tester.session_id[:20]}...")
    print(f"✅ Vault ID: {scim_tester.vault_id}")
    print(f"✅ Ready for SCIM testing")
else:
    print("❌ Authentication failed - cannot proceed with SCIM tests")
    
print("\n" + "=" * 60)

🔐 Testing authentication...
🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.31s)

🎉 Authentication Successful!
🆔 Session ID: 5DF6BFDAF1956FC4E5D8...
🏢 Vault ID: 92425

✅ Session established: 5DF6BFDAF1956FC4E5D8...
✅ Vault ID: 92425
✅ Ready for SCIM testing



## 🔍 Test SCIM Discovery

Testing SCIM discovery endpoints for provider configuration, resource types, and schemas.

In [7]:
# Test 02: SCIM discovery endpoints
print("🔍 Testing SCIM discovery endpoints...")
print("=" * 60)

scim_tester.test_scim_discovery()

print("\n📋 Discovery Summary:")
print(f"⚙️ Provider Config: {'✅ Available' if scim_tester.scim_provider_config else '❌ Not available'}")
print(f"📦 Resource Types: {len(scim_tester.scim_resource_types)} found")
print(f"📋 Schemas: {len(scim_tester.scim_schemas)} found")

print("\n" + "=" * 60)

🔍 Testing SCIM discovery endpoints...
✅ SCIM Provider Configuration: SUCCESS (0.23s)
✅ SCIM Resource Types: SUCCESS (0.25s)
✅ SCIM Schemas: SUCCESS (0.38s)

📋 Discovery Summary:
⚙️ Provider Config: ✅ Available
📦 Resource Types: 4 found
📋 Schemas: 6 found



## 👥 Test SCIM User and Group Management

Testing SCIM user and group management endpoints.

In [8]:
# Test 03: SCIM users
print("👥 Testing SCIM users...")
print("=" * 60)

scim_tester.test_scim_users()

print("\n" + "=" * 60)

👥 Testing SCIM users...
✅ SCIM Users Retrieval: SUCCESS (0.63s)
👥 Found 6 SCIM users



In [9]:
# Test 04: SCIM groups
print("👥 Testing SCIM groups...")
print("=" * 60)

scim_tester.test_scim_groups()

print("\n" + "=" * 60)

👥 Testing SCIM groups...
❌ SCIM Groups Retrieval: FAILED (0.16s) - Status: 404



## 🎯 Comprehensive Test Summary and Cleanup

Final summary of all SCIM API tests with comprehensive results analysis.

In [10]:
# Comprehensive Test Summary and Cleanup
print("🎯 SCIM API TESTING - FINAL SUMMARY")
print("=" * 60)

# Perform cleanup of any SCIM operations
cleanup_results = scim_tester.cleanup_scim_operations()

# Count results by status
success_count = sum(1 for result in scim_tester.results if result.success)
total_tests = len(scim_tester.results)
total_time = sum(result.response_time for result in scim_tester.results)

print("\n📊 TEST SUMMARY")
print("=" * 40)
print(f"✅ SUCCESS: {success_count}")
if total_tests > success_count:
    print(f"❌ FAILED: {total_tests - success_count}")
print(f"⏱️ Total time: {total_time:.2f}s")
print(f"🔐 Session: {'✅ Active' if scim_tester.session_id else '❌ Inactive'}")

# Detailed results analysis
print("\n📊 DETAILED RESULTS")
print("-" * 40)

for i, result in enumerate(scim_tester.results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    test_name = result.test_name or f"Test {i}"
    
    print(f"{status} [{i:02d}]: {test_name} ({result.response_time:.2f}s)")
    
    # Show additional details for interesting results
    if result.details:
        if 'total_users' in result.details:
            print(f"        👥 Users: {result.details['total_users']}")
        if 'total_groups' in result.details:
            print(f"        👥 Groups: {result.details['total_groups']}")
        if 'has_response_data' in result.details:
            print(f"        📄 Response Data: {'✅' if result.details['has_response_data'] else '❌'}")
    
    if not result.success and result.error_message:
        print(f"        ❌ Error: {result.error_message}")

# Final statistics
print("\n🏆 FINAL STATISTICS")
print("-" * 40)
print(f"📈 Success Rate: {success_count}/{total_tests} ({(success_count/total_tests*100):.1f}%)")
print(f"⏱️ Total Execution Time: {total_time:.2f}s")
print(f"⚙️ SCIM Provider Config: {'✅' if scim_tester.scim_provider_config else '❌'}")
print(f"📦 Resource Types: {len(scim_tester.scim_resource_types)}")
print(f"📋 Schemas: {len(scim_tester.scim_schemas)}")
print(f"👥 SCIM Users: {len(scim_tester.scim_users)}")
print(f"👥 SCIM Groups: {len(scim_tester.scim_groups)}")

# Vault state summary
print("\n🏢 VAULT STATE")
print("-" * 40)
print(f"🔗 Connected to: {scim_tester.vault_url}")
print(f"🆔 Vault ID: {scim_tester.vault_id}")
print(f"👤 User: {scim_tester.username}")
print(f"🔐 Session: {'Active' if scim_tester.session_id else 'None'}")

# API coverage summary
tested_endpoints = [
    "Authentication",
    "SCIM Provider Configuration",
    "SCIM Resource Types",
    "SCIM Schemas",
    "SCIM Users Retrieval",
    "SCIM Groups Retrieval"
]

print("\n🎯 API COVERAGE")
print("-" * 40)
for i, endpoint in enumerate(tested_endpoints, 1):
    status = "✅" if i <= len(scim_tester.results) else "⏸️"
    print(f"{status} {endpoint}")

print("\n✅ SCIM API testing completed!")
print("📋 All operations were read-only and SCIM-safe")
print("👥 No SCIM resources modified - vault integrity preserved")

# Export summary for next section
scim_summary = {
    'section': '13_scim',
    'total_tests': total_tests,
    'successful_tests': success_count,
    'total_time': total_time,
    'success_rate': (success_count/total_tests*100) if total_tests else 0,
    'scim_provider_config': bool(scim_tester.scim_provider_config),
    'scim_resource_types': len(scim_tester.scim_resource_types),
    'scim_schemas': len(scim_tester.scim_schemas),
    'scim_users': len(scim_tester.scim_users),
    'scim_groups': len(scim_tester.scim_groups),
    'vault_id': scim_tester.vault_id,
    'session_active': bool(scim_tester.session_id)
}

print(f"\n📦 Section summary exported for continuation: {scim_summary['success_rate']:.1f}% success rate")
print("=" * 60)

🎯 SCIM API TESTING - FINAL SUMMARY

📊 TEST SUMMARY
✅ SUCCESS: 5
❌ FAILED: 1
⏱️ Total time: 1.96s
🔐 Session: ✅ Active

📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.31s)
✅ PASS [02]: SCIM Provider Configuration (0.23s)
        📄 Response Data: ✅
✅ PASS [03]: SCIM Resource Types (0.25s)
        📄 Response Data: ✅
✅ PASS [04]: SCIM Schemas (0.38s)
        📄 Response Data: ✅
✅ PASS [05]: SCIM Users Retrieval (0.63s)
        👥 Users: 6
❌ FAIL [06]: SCIM Groups Retrieval (0.16s)
        ❌ Error: Status: 404

🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 5/6 (83.3%)
⏱️ Total Execution Time: 1.96s
⚙️ SCIM Provider Config: ✅
📦 Resource Types: 4
📋 Schemas: 6
👥 SCIM Users: 6
👥 SCIM Groups: 0

🏢 VAULT STATE
----------------------------------------
🔗 Connected to: https://vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425
👤 User: veevatools@vv-consulting.com
🔐 Session: Active

🎯 API COVERAGE
------------------